## AzureML Endpoint and Content Formatter Imports

In [18]:
from langchain_community.llms.azureml_endpoint import (
    AzureMLEndpointApiType,
    CustomOpenAIContentFormatter,
    ContentFormatterBase
)
from langchain_community.llms.azureml_endpoint import AzureMLOnlineEndpoint
from langchain_core.messages import HumanMessage

## Phi3CodeFormatter Class
This class defines a custom content formatter for the Phi-3 model, handling request and response formatting for AzureML endpoints.

In [10]:
from langchain_core.outputs import Generation
from typing import List, Dict
import json

class Phi3CodeFormattrer(ContentFormatterBase):
    @property
    def supported_api_types(self) -> List[AzureMLEndpointApiType]:
        return [ AzureMLEndpointApiType.serverless]
        
    def format_request_payload(
        self, prompt: str, model_kwargs: Dict, api_type: AzureMLEndpointApiType
    ) -> bytes:
        """Formats the request according to the chosen api"""
        prompt =  ContentFormatterBase.escape_special_characters(prompt)

        if api_type in [
            AzureMLEndpointApiType.serverless
        ]:
            request_payload = json.dumps({
                "messages": [{
                    "role": "user",
                    "content": prompt
                    
                }],
                **model_kwargs
            })
        else:
            raise ValueError(
                    f"`api_type` {api_type} is not supported by this formatter"
                )
        
        return str.encode(request_payload)


    def format_response_payload(
        self, output: bytes, api_type: AzureMLEndpointApiType
    ) -> Generation:
        try:
            output = json.loads(output)
            choice = output['choices'][0]
            content = choice['message']['content']
            return Generation(
                text=content.strip(),
                generation_info=dict(
                    finish_reason=choice.get("finish_reason"),
                    logprobs=choice.get("logprobs"),
                ),
            )
        except (KeyError, IndexError, TypeError) as e:
            raise ValueError(self.format_error_msg.format(api_type=api_type)) from e 
            
            
    
        

## AzureML Online Endpoint Configuration
This code configures an AzureML online endpoint for the Phi-3 model, specifying the URL, API type, key, content formatter, and model parameters.

In [11]:
llm = AzureMLOnlineEndpoint(
    endpoint_url="", # enter your endpoint    
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key="", # enter your api key
    content_formatter=Phi3CodeFormattrer(),
    model_kwargs={
        "temperature": 0.8, "max_new_tokens": 20
    },
    verbose=True,
)

## CrewAI Agent Configuration
This code sets up a CrewAI agent for web content scraping, defining its role, goal, backstory, and tools, using a custom LLM.

In [12]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, FileReadTool, WebsiteSearchTool

# scrape_tool = ScrapeWebsiteTool(website_url="https://techcrunch.com/2024/07/05/quantum-rise-grabs-15m-seed-for-its-ai-driven-consulting-2-0-startup/")
scrape_tool = ScrapeWebsiteTool()

web_content_scraper = Agent(
    role="Web Content Scraper",
    goal="Collect and synthesize detailed information",
    backstory="""You are a specialized agent designed to efficiently gather, 
    process, and compile information from various online sources. Your primary function is to 
    provide comprehensive, well-organized content on specified topics by leveraging web scraping 
    techniques and analytical skills""",
    verbose=True,
    allow_delegation=False,
    tools=[scrape_tool],
    llm=llm
)

In [13]:
summary_agent = Agent(
    role="Summary Agent",
    goal="Deliver concise and engaging 60-word summaries of news content",
    backstory="""
    As the Summary Agent, you are an expert in distilling complex information into brief, 
    impactful summaries. Your skill lies in identifying the core message of any news piece 
    and presenting it in a way that is both informative and captivating. You excel at using 
    precise language to convey maximum information in minimal words, ensuring that your summaries are comprehensive yet concise.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [14]:

web_content_scraper_task = Task(
    description=(
        """Extract and synthesize comprehensive information by scraping from the given site ({site})
        The final output should provide a comprehensive overview of the data, 
        with a focus on delivering well-organized and insightful content in under 500 words."""
    ),
    expected_output=(
        """
        Produce a document that provides a comprehensive overview of the scraped content, 
        identifying key themes, trends, and insights to create a cohesive and comprehensive narrative 
        in under 1000 words. The document should be well-written, provide valuable insights derived from 
        the collective information of all scraped sources, and ensure all information is accurately cited. 
        The narrative should flow logically between different points and sections, offering readers 
        a clear and insightful understanding of the website's content.
        """
    ),
    agent=web_content_scraper
)


summary_task = Task(
    description=(
        """Read the provided news content and create a compelling summary that captures 
        the essence of the story. Focus on the most crucial information, key points, and any 
        significant implications. Ensure the summary is clear, engaging, and informative despite 
        its brevity.
        """
    ),
    expected_output=(
        """Produce a final 100-word summary that effectively encapsulates the main points of the news story, 
        is concise yet comprehensive, engaging, and easy to understand.
        """
    ),
    agent=summary_agent
)


## Build Crew

In [16]:
tech_crew = Crew(
    agents=[web_content_scraper,
            summary_agent],
    
    tasks=[web_content_scraper_task,
           summary_task],
    verbose=2,
)

## Kickoff the agents

In [ ]:
# Example data for kicking off the process
tech_input = {
   "site": "https://www.livemint.com/technology/gadgets/amazon-deals-on-car-and-bike-accessories-choose-from-dashcams-vacuum-cleaners-pressure-washers-and-more-11720178118410.html",
}

result = tech_crew.kickoff(inputs=tech_input)

## Result

In [ ]:
from IPython.display import Markdown
Markdown(result)